In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
baixa_carga = pd.read_csv('./dataset_rotulado/SQL Server/monitoramento-memoria-baixa-carga-sqlserver.csv', sep=',')
media_carga = pd.read_csv('./dataset_rotulado/SQL Server/monitoramento-memoria-media-carga-sqlserver.csv', sep=',')
alta_carga = pd.read_csv('./dataset_rotulado/SQL Server/monitoramento-memoria-alta-carga-sqlserver.csv', sep=',')

In [ ]:
baixa_carga

In [ ]:
baixa_carga = baixa_carga.drop(columns=['Date', 'Time', 'trend', 'seasonal', 'residual'], errors='ignore')


In [ ]:
print(baixa_carga['Aging_Label'].value_counts(normalize=True))

Aging_Label
1.0    0.741751
0.0    0.258249
Name: proportion, dtype: float64


In [ ]:
X = baixa_carga.drop(columns=['Aging_Label'])
y = baixa_carga['Aging_Label']

kf = KFold(n_splits=5, shuffle=True)
accuracies = []
precisions = []
recalls = []
f1_scores = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='binary')
    rec = recall_score(y_test, y_pred, average='binary')
    f1 = f1_score(y_test, y_pred, average='binary')

    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1_scores.append(f1)

print("Acurácias em cada fold:", accuracies)
print("Precisões em cada fold:", precisions)
print("Revocações em cada fold:", recalls)
print("F1-scores em cada fold:", f1_scores)

print("\nMédias:")
print("Acurácia média:", sum(accuracies)/len(accuracies))
print("Precisão média:", sum(precisions)/len(precisions))
print("Revocação média:", sum(recalls)/len(recalls))
print("F1-score médio:", sum(f1_scores)/len(f1_scores))


Acurácias em cada fold: [1.0, 0.9997098084735926, 0.9988392338943702, 0.9994196169471852, 0.9994195327238427]
Precisões em cada fold: [1.0, 0.9996086105675147, 0.99921875, 0.999609756097561, 0.9998036135113905]
Revocações em cada fold: [1.0, 1.0, 0.99921875, 0.999609756097561, 0.9994110718492344]
F1-scores em cada fold: [1.0, 0.9998042669798395, 0.99921875, 0.999609756097561, 0.9996073041429413]

Médias:
Acurácia média: 0.9994776384077981
Precisão média: 0.9996481460352932
Revocação média: 0.9996479155893591
F1-score médio: 0.9996480154440684


# Avaliar o Modelo de Referência por carga

In [ ]:
resultados = pd.DataFrame(columns=['Modelo', 'Carga Predita', 'Acurácia', 'Precisão', 'Revocação', 'F1-Score'])

media_acuracia_media = sum(accuracies)/len(accuracies)
media_precisao_media = sum(precisions)/len(precisions)
media_revocacao_media = sum(recalls)/len(recalls)
media_f1_media = sum(f1_scores)/len(f1_scores)

resultados.loc[len(resultados)] = [
	f"{clf.__class__.__name__}-Baixa Carga",
    "Baixa",
    media_acuracia_media,
    media_precisao_media,
    media_revocacao_media,
    media_f1_media
]

In [ ]:
media_carga = media_carga.drop(columns=['Date', 'Time', 'trend', 'seasonal', 'residual'], errors='ignore')

X2 = media_carga.drop(columns=['Aging_Label'], errors='ignore')
y2 = media_carga['Aging_Label']

X2 = X2[X.columns]

X2_scaled = X2

y2_pred = clf.predict(X2_scaled)

print("\nAvaliação em df2:")
print("Acurácia:", accuracy_score(y2, y2_pred))
print("Precisão:", precision_score(y2, y2_pred, average='binary'))
print("Revocação:", recall_score(y2, y2_pred, average='binary'))
print("F1-score:", f1_score(y2, y2_pred, average='binary'))


Avaliação em df2:
Acurácia: 0.6541833594611857
Precisão: 0.7626029400446395
Revocação: 0.7753129890453834
F1-score: 0.7689054436813719


In [ ]:
media_acuracia_media = accuracy_score(y2, y2_pred)
media_precisao_media = precision_score(y2, y2_pred, average='binary')
media_revocacao_media = recall_score(y2, y2_pred, average='binary')
media_f1_media = f1_score(y2, y2_pred, average='binary')

resultados.loc[len(resultados)] = [
	f"{clf.__class__.__name__}-Baixa Carga",
	"Média",
	media_acuracia_media,
	media_precisao_media,
	media_revocacao_media,
	media_f1_media
]

In [ ]:

alta_carga = alta_carga.drop(columns=['Date', 'Time', 'trend', 'seasonal', 'residual'], errors='ignore')

X3 = alta_carga.drop(columns=['Aging_Label'], errors='ignore')
y3 = alta_carga['Aging_Label']

X3 = X3[X.columns]

X3_scaled = X3

y3_pred = clf.predict(X3_scaled)

print("\nAvaliação em df3:")
print("Acurácia:", accuracy_score(y3, y3_pred))
print("Precisão:", precision_score(y3, y3_pred, average='binary'))
print("Revocação:", recall_score(y3, y3_pred, average='binary'))
print("F1-score:", f1_score(y3, y3_pred, average='binary'))


Avaliação em df3:
Acurácia: 0.7355801329501582
Precisão: 0.8028337119316213
Revocação: 0.8394524959742351
F1-score: 0.8207348513175762


In [ ]:
media_acuracia_media = accuracy_score(y3, y3_pred)
media_precisao_media = precision_score(y3, y3_pred, average='binary')
media_revocacao_media = recall_score(y3, y3_pred, average='binary')
media_f1_media = f1_score(y3, y3_pred, average='binary')

resultados.loc[len(resultados)] = [
	f"{clf.__class__.__name__}-Baixa Carga",
	"Alta",
	media_acuracia_media,
	media_precisao_media,
	media_revocacao_media,
	media_f1_media
]

In [13]:
resultados

,Modelo,Carga Predita,Acurácia,Precisão,Revocação,F1-Score
0,RandomForestClassifier-Baixa Carga,Baixa,0.999478,0.999648,0.999648,0.999648
1,RandomForestClassifier-Baixa Carga,Média,0.654183,0.762603,0.775313,0.768905
2,RandomForestClassifier-Baixa Carga,Alta,0.735580,0.802834,0.839452,0.820735


In [14]:
print(resultados)

                               Modelo Carga Predita  Acurácia  Precisão  \
0  RandomForestClassifier-Baixa Carga         Baixa  0.999478  0.999648   
1  RandomForestClassifier-Baixa Carga         Média  0.654183  0.762603   
2  RandomForestClassifier-Baixa Carga          Alta  0.735580  0.802834   

   Revocação  F1-Score  
0   0.999648  0.999648  
1   0.775313  0.768905  
2   0.839452  0.820735  


# Simular Drift Súbito

In [15]:
df_baixa_inicio = baixa_carga[baixa_carga['Elapsed_time'] <= 86145]
df_media_fim = media_carga[media_carga['Elapsed_time'] >= 86150]
df_baixa_media = pd.concat([df_baixa_inicio, df_media_fim], ignore_index=True)
df_baixa_media.reset_index(drop=True, inplace=True)


## Avaliação do modelo de referência no cenário de Drift Súbito (Baixo-Médio):

In [ ]:
X_baixa_media = df_baixa_media.drop(columns=['Aging_Label'], errors='ignore')
y_baixa_media = df_baixa_media['Aging_Label']

X_baixa_media = X_baixa_media[X.columns]

y_baixa_media_pred = clf.predict(X_baixa_media)

print("Avaliação em df_baixa_media:")
print("Acurácia:", accuracy_score(y_baixa_media, y_baixa_media_pred))
print("Precisão:", precision_score(y_baixa_media, y_baixa_media_pred, average='binary'))
print("Revocação:", recall_score(y_baixa_media, y_baixa_media_pred, average='binary'))
print("F1-score:", f1_score(y_baixa_media, y_baixa_media_pred, average='binary'))


Avaliação em df_baixa_media:
Acurácia: 0.7342797422051908
Precisão: 0.8521056512122132
Revocação: 0.7738085860575029
F1-score: 0.8110718929964704


In [ ]:
X_baixa_media = df_baixa_media.drop(columns=['Aging_Label'], errors='ignore')
y_baixa_media = df_baixa_media['Aging_Label']

X_baixa_media = X_baixa_media[X.columns]

y_baixa_media_pred = clf.predict(X_baixa_media)

resultados.loc[len(resultados)] = [
	f"{clf.__class__.__name__}-Baixa Carga",
    "Baixa-Média",
    accuracy_score(y_baixa_media, y_baixa_media_pred),
    precision_score(y_baixa_media, y_baixa_media_pred, average='binary'),
    recall_score(y_baixa_media, y_baixa_media_pred, average='binary'),
    f1_score(y_baixa_media, y_baixa_media_pred, average='binary')
]

In [ ]:
df_baixa_inicio = baixa_carga[baixa_carga['Elapsed_time'] <= 86145]
df_alta_fim = alta_carga[alta_carga['Elapsed_time'] >= 86150]
df_baixa_alta = pd.concat([df_baixa_inicio, df_alta_fim], ignore_index=True)
df_baixa_alta.reset_index(drop=True, inplace=True)

## Avaliação do modelo de referência no cenário de Drift Súbito (Baixo-Alta):

In [ ]:
X_baixa_alta = df_baixa_alta.drop(columns=['Aging_Label'], errors='ignore')
y_baixa_alta = df_baixa_alta['Aging_Label']

X_baixa_alta = X_baixa_alta[X.columns]

y_baixa_alta_pred = clf.predict(X_baixa_alta)

print("Avaliação em df_baixa_alta:")
print("Acurácia:", accuracy_score(y_baixa_alta, y_baixa_alta_pred))
print("Precisão:", precision_score(y_baixa_alta, y_baixa_alta_pred, average='binary'))
print("Revocação:", recall_score(y_baixa_alta, y_baixa_alta_pred, average='binary'))
print("F1-score:", f1_score(y_baixa_alta, y_baixa_alta_pred, average='binary'))

Avaliação em df_baixa_alta:
Acurácia: 0.8218525936892217
Precisão: 0.9055818189703777
Revocação: 0.8405797101449275
F1-score: 0.8718708896173037


In [20]:
resultados.loc[len(resultados)] = [
	f"{clf.__class__.__name__}-Baixa Carga",
    "Baixa-Alta",
    accuracy_score(y_baixa_alta, y_baixa_alta_pred),
    precision_score(y_baixa_alta, y_baixa_alta_pred, average='binary'),
    recall_score(y_baixa_alta, y_baixa_alta_pred, average='binary'),
    f1_score(y_baixa_alta, y_baixa_alta_pred, average='binary')
]

In [21]:
print(resultados)

                               Modelo Carga Predita  Acurácia  Precisão  \
0  RandomForestClassifier-Baixa Carga         Baixa  0.999478  0.999648   
1  RandomForestClassifier-Baixa Carga         Média  0.654183  0.762603   
2  RandomForestClassifier-Baixa Carga          Alta  0.735580  0.802834   
3  RandomForestClassifier-Baixa Carga   Baixa-Média  0.734280  0.852106   
4  RandomForestClassifier-Baixa Carga    Baixa-Alta  0.821853  0.905582   

   Revocação  F1-Score  
0   0.999648  0.999648  
1   0.775313  0.768905  
2   0.839452  0.820735  
3   0.773809  0.811072  
4   0.840580  0.871871  


# Simular Drift Gradual

In [ ]:
stable_period_size = 5000
transition_period_size = 10000 
random_seed = 42

np.random.seed(random_seed)

iter_baixa = baixa_carga.iterrows()
iter_media = media_carga.iterrows()
iter_alta = alta_carga.iterrows()

stream_rows = []

print("Iniciando a criação do stream de drift gradual (versão série temporal)...")

try:
    print(f"Fase 1: Adicionando {stable_period_size} amostras de carga baixa...")
    for _ in range(stable_period_size):
        stream_rows.append(next(iter_baixa)[1])

    print(f"Fase 2: Transição de Baixa -> Média em {transition_period_size} amostras...")
    for i in range(transition_period_size):
        p_media = i / transition_period_size
        if np.random.rand() < p_media:
            stream_rows.append(next(iter_media)[1])
        else:
            stream_rows.append(next(iter_baixa)[1])

    print(f"Fase 3: Adicionando {stable_period_size} amostras de carga média...")
    for _ in range(stable_period_size):
        stream_rows.append(next(iter_media)[1])

    print(f"Fase 4: Transição de Média -> Alta em {transition_period_size} amostras...")
    for i in range(transition_period_size):
        p_alta = i / transition_period_size
        if np.random.rand() < p_alta:
            stream_rows.append(next(iter_alta)[1])
        else:
            stream_rows.append(next(iter_media)[1])

    print(f"Fase 5: Adicionando {stable_period_size} amostras de carga alta...")
    for _ in range(stable_period_size):
        stream_rows.append(next(iter_alta)[1])

except StopIteration:
    print("Aviso: Um dos dataframes de carga foi totalmente consumido.")

df_gradual_drift_ts = pd.DataFrame(stream_rows)

df_gradual_drift_ts['Elapsed_time'] = np.arange(len(df_gradual_drift_ts)) * 5
df_gradual_drift_ts.reset_index(drop=True, inplace=True)


print("\\nStream de drift gradual (série temporal) criado com sucesso!")
print(f"Tamanho total do dataframe: {df_gradual_drift_ts.shape[0]} amostras.")

Iniciando a criação do stream de drift gradual (versão série temporal)...
Fase 1: Adicionando 5000 amostras de carga baixa...
Fase 2: Transição de Baixa -> Média em 10000 amostras...
Fase 3: Adicionando 5000 amostras de carga média...
Fase 4: Transição de Média -> Alta em 10000 amostras...
Fase 5: Adicionando 5000 amostras de carga alta...
\nStream de drift gradual (série temporal) criado com sucesso!
Tamanho total do dataframe: 35000 amostras.


## Avaliação do modelo de referência no cenário de Drift Gradual:

In [ ]:
X_gradual_ts = df_gradual_drift_ts.drop(columns=['Aging_Label'], errors='ignore')
y_gradual_ts = df_gradual_drift_ts['Aging_Label']

X_gradual_ts = X_gradual_ts[X.columns]

y_gradual_ts_pred = clf.predict(X_gradual_ts)

print("\\nAvaliação do modelo de referência no cenário de Drift Gradual (Série Temporal):")
acc_gradual_ts = accuracy_score(y_gradual_ts, y_gradual_ts_pred)
prec_gradual_ts = precision_score(y_gradual_ts, y_gradual_ts_pred, average='binary')
rec_gradual_ts = recall_score(y_gradual_ts, y_gradual_ts_pred, average='binary')
f1_gradual_ts = f1_score(y_gradual_ts, y_gradual_ts_pred, average='binary')

print(f"Acurácia: {acc_gradual_ts}")
print(f"Precisão: {prec_gradual_ts}")
print(f"Revocação: {rec_gradual_ts}")
print(f"F1-score: {f1_gradual_ts}")


\nAvaliação do modelo de referência no cenário de Drift Gradual (Série Temporal):
Acurácia: 0.7792285714285714
Precisão: 0.9219776134814753
Revocação: 0.7926846590909091
F1-score: 0.8524565121918619


In [24]:
resultados.loc[len(resultados)] = [
	f"{clf.__class__.__name__}-Baixa Carga",
    "Drift Gradual", # TS = Time-Series
    acc_gradual_ts,
    prec_gradual_ts,
    rec_gradual_ts,
    f1_gradual_ts
]

# Exibir a tabela de resultados atualizada
print("\\n--- Tabela de Resultados Atualizada ---")
print(resultados)

\n--- Tabela de Resultados Atualizada ---
                               Modelo  Carga Predita  Acurácia  Precisão  \
0  RandomForestClassifier-Baixa Carga          Baixa  0.999478  0.999648   
1  RandomForestClassifier-Baixa Carga          Média  0.654183  0.762603   
2  RandomForestClassifier-Baixa Carga           Alta  0.735580  0.802834   
3  RandomForestClassifier-Baixa Carga    Baixa-Média  0.734280  0.852106   
4  RandomForestClassifier-Baixa Carga     Baixa-Alta  0.821853  0.905582   
5  RandomForestClassifier-Baixa Carga  Drift Gradual  0.779229  0.921978   

   Revocação  F1-Score  
0   0.999648  0.999648  
1   0.775313  0.768905  
2   0.839452  0.820735  
3   0.773809  0.811072  
4   0.840580  0.871871  
5   0.792685  0.852457  


# Simular Drift Recorrene

In [ ]:
import pandas as pd
import numpy as np

period_length = 5800

num_cycles = 2

random_seed = 42
np.random.seed(random_seed)

iter_baixa_rec = baixa_carga.iterrows()
iter_media_rec = media_carga.iterrows()
iter_alta_rec = alta_carga.iterrows()

stream_rows_rec = []

print("Iniciando a criação do stream de drift recorrente...")

try:
    for cycle in range(num_cycles):
        print(f"Iniciando ciclo {cycle + 1}/{num_cycles}...")
        
        print(f"  Adicionando {period_length} amostras de carga baixa...")
        for _ in range(period_length):
            stream_rows_rec.append(next(iter_baixa_rec)[1])

        print(f"  Adicionando {period_length} amostras de carga média...")
        for _ in range(period_length):
            stream_rows_rec.append(next(iter_media_rec)[1])
            
        print(f"  Adicionando {period_length} amostras de carga alta...")
        for _ in range(period_length):
            stream_rows_rec.append(next(iter_alta_rec)[1])

except StopIteration:
    print("Aviso: Um dos dataframes de carga foi totalmente consumido antes do fim dos ciclos.")

df_recurrent_drift = pd.DataFrame(stream_rows_rec)
df_recurrent_drift['Elapsed_time'] = np.arange(len(df_recurrent_drift)) * 5
df_recurrent_drift.reset_index(drop=True, inplace=True)

print("\\nStream de drift recorrente criado com sucesso!")
print(f"Tamanho total do dataframe: {df_recurrent_drift.shape[0]} amostras.")

Iniciando a criação do stream de drift recorrente...
Iniciando ciclo 1/2...
  Adicionando 5800 amostras de carga baixa...
  Adicionando 5800 amostras de carga média...
  Adicionando 5800 amostras de carga alta...
Iniciando ciclo 2/2...
  Adicionando 5800 amostras de carga baixa...
  Adicionando 5800 amostras de carga média...
  Adicionando 5800 amostras de carga alta...
\nStream de drift recorrente criado com sucesso!
Tamanho total do dataframe: 34800 amostras.


## Avaliação do modelo de referência no cenário de Drift Recorrente:

In [ ]:
X_recurrent = df_recurrent_drift.drop(columns=['Aging_Label'], errors='ignore')
y_recurrent = df_recurrent_drift['Aging_Label']
X_recurrent = X_recurrent[X.columns]

y_recurrent_pred = clf.predict(X_recurrent)

print("\\nAvaliação do modelo de referência no cenário de Drift Recorrente:")
acc_recurrent = accuracy_score(y_recurrent, y_recurrent_pred)
prec_recurrent = precision_score(y_recurrent, y_recurrent_pred, average='binary')
rec_recurrent = recall_score(y_recurrent, y_recurrent_pred, average='binary')
f1_recurrent = f1_score(y_recurrent, y_recurrent_pred, average='binary')

print(f"Acurácia: {acc_recurrent}")
print(f"Precisão: {prec_recurrent}")
print(f"Revocação: {rec_recurrent}")
print(f"F1-score: {f1_recurrent}")


\nAvaliação do modelo de referência no cenário de Drift Recorrente:
Acurácia: 0.580948275862069
Precisão: 0.7917983963344788
Revocação: 0.6325402635431918
F1-score: 0.7032658459660189


In [ ]:

resultados.loc[len(resultados)] = [
	f"{clf.__class__.__name__}-Baixa Carga",
    "Drift Recorrente",
    acc_recurrent,
    prec_recurrent,
    rec_recurrent,
    f1_recurrent
]

print("\\n--- Tabela de Resultados Final ---")
print(resultados)

\n--- Tabela de Resultados Final ---
                               Modelo     Carga Predita  Acurácia  Precisão  \
0  RandomForestClassifier-Baixa Carga             Baixa  0.999478  0.999648   
1  RandomForestClassifier-Baixa Carga             Média  0.654183  0.762603   
2  RandomForestClassifier-Baixa Carga              Alta  0.735580  0.802834   
3  RandomForestClassifier-Baixa Carga       Baixa-Média  0.734280  0.852106   
4  RandomForestClassifier-Baixa Carga        Baixa-Alta  0.821853  0.905582   
5  RandomForestClassifier-Baixa Carga     Drift Gradual  0.779229  0.921978   
6  RandomForestClassifier-Baixa Carga  Drift Recorrente  0.580948  0.791798   

   Revocação  F1-Score  
0   0.999648  0.999648  
1   0.775313  0.768905  
2   0.839452  0.820735  
3   0.773809  0.811072  
4   0.840580  0.871871  
5   0.792685  0.852457  
6   0.632540  0.703266  


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from river.drift.binary import DDM

def executar_fluxo_com_ddm_fit(modelo_base, df_stream, tipo, label_col='Aging_Label', window_size=2000):

    X_stream = df_stream.drop(columns=[label_col])
    y_stream = df_stream[label_col]

    ddm = DDM()

    y_preds = []
    ocorrencias_drift = []
    data_window = []

    print("Iniciando simulação de fluxo de dados com detecção de drift...")

    for i in range(len(df_stream)):
        x_atual = X_stream.iloc[[i]]
        y_verdadeiro = y_stream.iloc[i]

        y_pred = modelo_base.predict(x_atual)[0]
        y_preds.append(y_pred)

        erro = int(y_pred != y_verdadeiro)

        data_window.append((x_atual, y_verdadeiro))
        if len(data_window) > window_size:
            data_window.pop(0)

        ddm.update(erro)

        if ddm.drift_detected:
            print(f"\n--- Drift Detectado no índice: {i} ---")
            ocorrencias_drift.append(i)

            X_window = pd.concat([x for x, _ in data_window])
            y_window = pd.Series([y for _, y in data_window])

            if len(np.unique(y_window)) > 1:
                modelo_base.fit(X_window, y_window)
            else:
                print(f"Janela de dados no índice {i} tem apenas uma classe. Ignorando re-treinamento.")

            ddm = DDM()
            print(f"Re-treinamento com {len(data_window)} amostras. DDM resetado.")

    acc = accuracy_score(y_stream, y_preds)
    f1 = f1_score(y_stream, y_preds, average='binary')
    prec = precision_score(y_stream,y_preds, average='binary')
    rec = recall_score(y_stream,y_preds, average='binary')

    print("\n--- Simulação Concluída ---")
    print(f"Total de drifts detectados: {len(ocorrencias_drift)}")
    print(f"Acurácia: {acc:.4f}")
    print(f"Precisão: {prec:.4f}")
    print(f"Revocação: {rec:.4f}")
    print(f"F1-score: {f1:.4f}")

    
    resultados.loc[len(resultados)] = [
        f"{modelo_base.__class__.__name__} - DDM - re_fit - Baixa Carga",
    f"Drift {tipo}",
    acc,
    prec,
    rec,
    f1
]

    return [modelo_base, resultados]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from river.drift import ADWIN

def executar_fluxo_com_adwin_fit(modelo_base, df_stream, tipo, label_col='Aging_Label', window_size=2000):

    X_stream = df_stream.drop(columns=[label_col])
    y_stream = df_stream[label_col]

    adwin = ADWIN()

    y_preds = []
    ocorrencias_drift = []
    data_window = []

    print(f"Iniciando simulação de fluxo de dados com ADWIN para drift do tipo '{tipo}'...")

    for i in range(len(df_stream)):
        x_atual = X_stream.iloc[[i]]
        y_verdadeiro = y_stream.iloc[i]

        y_pred = modelo_base.predict(x_atual)[0]
        y_preds.append(y_pred)

        erro = int(y_pred != y_verdadeiro)

        data_window.append((x_atual, y_verdadeiro))
        if len(data_window) > window_size:
            data_window.pop(0)

        adwin.update(erro)

        if adwin.drift_detected:
            print(f"\n--- Drift Detectado (ADWIN) no índice: {i} ---")
            ocorrencias_drift.append(i)

            X_window = pd.concat([x for x, _ in data_window])
            y_window = pd.Series([y for _, y in data_window])

            if len(np.unique(y_window)) > 1:
                print(f"Re-treinando o modelo com as últimas {len(data_window)} amostras...")
                modelo_base.fit(X_window, y_window)
            else:
                print(f"Janela de dados no índice {i} tem apenas uma classe. Ignorando re-treinamento.")

            adwin = ADWIN()
            print("Detector ADWIN resetado.")

    acc = accuracy_score(y_stream, y_preds)
    f1 = f1_score(y_stream, y_preds, average='binary')
    prec = precision_score(y_stream,y_preds, average='binary')
    rec = recall_score(y_stream,y_preds, average='binary')

    print("\n--- Simulação Concluída (ADWIN) ---")
    print(f"Total de drifts detectados: {len(ocorrencias_drift)}")
    print(f"Acurácia: {acc:.4f}")
    print(f"Precisão: {prec:.4f}")
    print(f"Revocação: {rec:.4f}")
    print(f"F1-score: {f1:.4f}")

    resultados.loc[len(resultados)] = [
        f"{modelo_base.__class__.__name__} - ADWIN - re_fit - Baixa Carga",
        f"Drift {tipo}",
        acc,
        prec,
        rec,
        f1
    ]

    return [modelo_base, resultados]

In [ ]:
novo_modelo, resultados = executar_fluxo_com_ddm_fit(clf, df_gradual_drift_ts, tipo='Gradual', label_col='Aging_Label', window_size=2000)
novo_modelo, resultados = executar_fluxo_com_ddm_fit(clf, df_recurrent_drift, tipo='Recurrente', label_col='Aging_Label', window_size=2000)
novo_modelo, resultados = executar_fluxo_com_ddm_fit(clf, df_baixa_alta, tipo='Drift Súbito (Baixa-Alta)', label_col='Aging_Label', window_size=2000)
novo_modelo, resultados = executar_fluxo_com_ddm_fit(clf, df_baixa_media, tipo='Drift Súbito (Baixa-Média)', label_col='Aging_Label', window_size=2000)

Iniciando simulação de fluxo de dados com detecção de drift...

--- Drift Detectado no índice: 6190 ---
Re-treinamento com 2000 amostras. DDM resetado.

--- Drift Detectado no índice: 6335 ---
Re-treinamento com 2000 amostras. DDM resetado.

--- Drift Detectado no índice: 6768 ---
Re-treinamento com 2000 amostras. DDM resetado.

--- Drift Detectado no índice: 11140 ---
Re-treinamento com 2000 amostras. DDM resetado.

--- Drift Detectado no índice: 20906 ---
Re-treinamento com 2000 amostras. DDM resetado.

--- Drift Detectado no índice: 21614 ---
Re-treinamento com 2000 amostras. DDM resetado.

--- Drift Detectado no índice: 22629 ---
Re-treinamento com 2000 amostras. DDM resetado.

--- Drift Detectado no índice: 23326 ---
Re-treinamento com 2000 amostras. DDM resetado.

--- Drift Detectado no índice: 27732 ---
Re-treinamento com 2000 amostras. DDM resetado.

--- Drift Detectado no índice: 29875 ---
Re-treinamento com 2000 amostras. DDM resetado.

--- Drift Detectado no índice: 30384 --

In [44]:
executar_fluxo_com_adwin_fit(clf, df_gradual_drift_ts, tipo='Gradual', label_col='Aging_Label')
executar_fluxo_com_adwin_fit(clf, df_recurrent_drift, tipo='Recurrente', label_col='Aging_Label')
executar_fluxo_com_adwin_fit(clf, df_baixa_alta, tipo='Drift Súbito (Baixa-Alta)', label_col='Aging_Label')
executar_fluxo_com_adwin_fit(clf, df_baixa_media, tipo='Drift Súbito (Baixa-Média)', label_col='Aging_Label')   

Iniciando simulação de fluxo de dados com ADWIN para drift do tipo 'Gradual'...

--- Drift Detectado (ADWIN) no índice: 159 ---
Re-treinando o modelo com as últimas 160 amostras...
Detector ADWIN resetado.

--- Drift Detectado (ADWIN) no índice: 3039 ---
Janela de dados no índice 3039 tem apenas uma classe. Ignorando re-treinamento.
Detector ADWIN resetado.

--- Drift Detectado (ADWIN) no índice: 4831 ---
Re-treinando o modelo com as últimas 2000 amostras...
Detector ADWIN resetado.

--- Drift Detectado (ADWIN) no índice: 4927 ---
Re-treinando o modelo com as últimas 2000 amostras...
Detector ADWIN resetado.

--- Drift Detectado (ADWIN) no índice: 6367 ---
Re-treinando o modelo com as últimas 2000 amostras...
Detector ADWIN resetado.

--- Drift Detectado (ADWIN) no índice: 7359 ---
Re-treinando o modelo com as últimas 2000 amostras...
Detector ADWIN resetado.

--- Drift Detectado (ADWIN) no índice: 11167 ---
Re-treinando o modelo com as últimas 2000 amostras...
Detector ADWIN resetado.

[RandomForestClassifier(),
                                                Modelo  \
 0                  RandomForestClassifier-Baixa Carga   
 1                  RandomForestClassifier-Baixa Carga   
 2                  RandomForestClassifier-Baixa Carga   
 3                  RandomForestClassifier-Baixa Carga   
 4                  RandomForestClassifier-Baixa Carga   
 5                  RandomForestClassifier-Baixa Carga   
 6                  RandomForestClassifier-Baixa Carga   
 7   RandomForestClassifier - DDM - re_fit - Baixa ...   
 8   RandomForestClassifier - DDM - re_fit - Baixa ...   
 9   RandomForestClassifier - DDM - re_fit - Baixa ...   
 10  RandomForestClassifier - DDM - re_fit - Baixa ...   
 11  RandomForestClassifier - ADWIN - re_fit - Baix...   
 12  RandomForestClassifier - ADWIN - re_fit - Baix...   
 13  RandomForestClassifier - ADWIN - re_fit - Baix...   
 14  RandomForestClassifier - ADWIN - re_fit - Baix...   
 
                        Carga Predita  Acur

In [45]:
resultados

,Modelo,Carga Predita,Acurácia,Precisão,Revocação,F1-Score
0,RandomForestClassifier-Baixa Carga,Baixa,0.999478,0.999648,0.999648,0.999648
1,RandomForestClassifier-Baixa Carga,Média,0.654183,0.762603,0.775313,0.768905
2,RandomForestClassifier-Baixa Carga,Alta,0.735580,0.802834,0.839452,0.820735
3,RandomForestClassifier-Baixa Carga,Baixa-Média,0.734280,0.852106,0.773809,0.811072
4,RandomForestClassifier-Baixa Carga,Baixa-Alta,0.821853,0.905582,0.840580,0.871871
5,RandomForestClassifier-Baixa Carga,Drift Gradual,0.779229,0.921978,0.792685,0.852457
6,RandomForestClassifier-Baixa Carga,Drift Recorrente,0.580948,0.791798,0.632540,0.703266
7,RandomForestClassifier - DDM - re_fit - Baixa ...,Drift Gradual,0.886257,0.919527,0.940980,0.930130
8,RandomForestClassifier - DDM - re_fit - Baixa ...,Drift Recurrente,0.613879,0.855847,0.611091,0.713050
9,RandomForestClassifier - DDM - re_fit - Baixa ...,Drift Drift Súbito (Baixa-Alta),0.801126,0.856232,0.870330,0.863223


In [46]:
print(resultados)

                                               Modelo  \
0                  RandomForestClassifier-Baixa Carga   
1                  RandomForestClassifier-Baixa Carga   
2                  RandomForestClassifier-Baixa Carga   
3                  RandomForestClassifier-Baixa Carga   
4                  RandomForestClassifier-Baixa Carga   
5                  RandomForestClassifier-Baixa Carga   
6                  RandomForestClassifier-Baixa Carga   
7   RandomForestClassifier - DDM - re_fit - Baixa ...   
8   RandomForestClassifier - DDM - re_fit - Baixa ...   
9   RandomForestClassifier - DDM - re_fit - Baixa ...   
10  RandomForestClassifier - DDM - re_fit - Baixa ...   
11  RandomForestClassifier - ADWIN - re_fit - Baix...   
12  RandomForestClassifier - ADWIN - re_fit - Baix...   
13  RandomForestClassifier - ADWIN - re_fit - Baix...   
14  RandomForestClassifier - ADWIN - re_fit - Baix...   

                       Carga Predita  Acurácia  Precisão  Revocação  F1-Score  
0      